In [ ]:
import os
import json
import base64
import pandas as pd
from typing import List, Dict


In [ ]:
import pandas as pd


In [ ]:
# -----------------------------------------------------------
# IBM Cloud / watsonx.ai setup
# -----------------------------------------------------------
# This notebook can run in two modes:
# 1) With watsonx.ai credentials (calls IBM Cloud)
# 2) Without credentials (falls back to the original stub outputs)
#
# Required env vars for IBM Cloud mode:
#   WATSONX_APIKEY      - watsonx.ai Runtime API key
#   WATSONX_PROJECT_ID  - watsonx.ai project ID
# Optional:
#   WATSONX_URL         - e.g., https://us-south.ml.cloud.ibm.com
#   WATSONX_TEXT_MODEL  - e.g., ibm/granite-3-8b-instruct
#   WATSONX_VISION_MODEL- e.g., meta-llama/llama-3-2-90b-vision-instruct
# -----------------------------------------------------------

%pip install -U "ibm_watsonx_ai>=1.1.14" -q

import os

WATSONX_APIKEY = os.getenv("Enter your newly created API key", "")
WATSONX_PROJECT_ID = os.getenv("Enter the project ID of your sandbox project", "")
WATSONX_URL = os.getenv("WATSONX_URL", "https://us-south.ml.cloud.ibm.com")

# Sensible defaults (override with env vars if you want)
WATSONX_TEXT_MODEL = os.getenv("WATSONX_TEXT_MODEL", "ibm/granite-3-8b-instruct")
WATSONX_VISION_MODEL = os.getenv("WATSONX_VISION_MODEL", "meta-llama/llama-3-2-90b-vision-instruct")

def _watsonx_ready() -> bool:
    return bool(WATSONX_APIKEY.strip()) and bool(WATSONX_PROJECT_ID.strip())

print("watsonx.ai ready:", _watsonx_ready())


In [ ]:
import os


In [ ]:
import os
import base64
import pandas as pd
from typing import List, Dict


In [ ]:
# -----------------------------------------------------------
# Cell 4: Load and encode gym equipment images
# -----------------------------------------------------------
# In this step, we prepare one or more images that show the
# gym equipment available to the user.
#
# The Vision Model cannot accept raw image files. It requires:
#   1) The image to be read as raw bytes (binary)
#   2) Then encoded as a base64 text string
#
# The base64 version is safe to send over the network inside
# the JSON request to the vision model.
#
# Students will update the file paths below to point to THEIR
# own images located on their computer.
# -----------------------------------------------------------


# 👇 Update these paths to point to the actual equipment images.
#    You can test with one image, then add two or three more.
#    If a file is missing, the code will safely raise a helpful error.
#    If you have a PC, it might look like this: "C:\Users\armenpischdotchian\download1.jpeg"
#    If you have a MAC, might look like this: "/Users/armenpischdotchian/Desktop/download1.jpeg"
#    The file path below is based on my settings, your's is different.

image_filepaths = [
    r"/Users/armenpischdotchian/Desktop/download1.jpeg",
    r"/Users/armenpischdotchian/Desktop/download3.jpeg",
    r"/Users/armenpischdotchian/Desktop/download2.jpeg",
]

# These lists will store:
#   - base64-encoded versions of the images
#   - the corresponding filenames
images = []
filenames = []

# -----------------------------------------------------------
# Loop through each image path:
#   - Check if the file exists
#   - Read it in binary mode
#   - Convert ("encode") the bytes into a base64 string
#
# The base64 string can be transmitted to the vision model,
# which will decode it internally and visually interpret it.
# -----------------------------------------------------------
for path in image_filepaths:

    # 1. Verify that the file path exists on the system
    if not os.path.exists(path):
        raise FileNotFoundError(
            f"The file was not found: {path}\n"
            "→ Make sure your file paths are correct!"
        )

    # 2. Open the file in "rb" mode (rb = read binary)
    with open(path, "rb") as f:
        # Read all bytes, then convert to base64, then decode to UTF-8 text
        encoded = base64.b64encode(f.read()).decode("utf-8")

    # 3. Append the encoded image and its raw filename
    images.append(encoded)
    filenames.append(os.path.basename(path))

# -----------------------------------------------------------
# Print confirmation that the images were loaded successfully.
# This helps students debug file path issues early.
# -----------------------------------------------------------
print(f"Loaded {len(images)} image(s):", filenames)


In [ ]:
import pandas as pd

def vision_model_describe(image_b64: str) -> str:
    """Describe a gym equipment image.

    Returns EXACT pipe-delimited format:
      description | category | workout_type

    If watsonx.ai credentials are not provided, falls back to the original stub
    output to keep notebook behavior/output consistent.
    """
    # --- Fallback (keeps the original notebook output) ---
    fallback = "Dumbbells and flat bench visible | strength_training | upper_body"

    if not _watsonx_ready():
        return fallback

    try:
        from ibm_watsonx_ai import Credentials
        from ibm_watsonx_ai.foundation_models import ModelInference
    except Exception:
        return fallback

    credentials = Credentials(url=WATSONX_URL, api_key=WATSONX_APIKEY)

    model = ModelInference(
        model_id=WATSONX_VISION_MODEL,
        credentials=credentials,
        project_id=WATSONX_PROJECT_ID,
        params={"max_tokens": 100},
    )

    system = (
        "You are a safety-conscious personal trainer assistant. "
        "Look at the image and identify the gym equipment shown. "
        "Return ONLY: description | category | workout_type. "
        "category must be one of: strength_training, cardio, flexibility, other. "
        "workout_type must be one of: upper_body, lower_body, full_body, core, cardio."
    )

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": system},
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{image_b64}"},
                },
            ],
        }
    ]

    try:
        resp = model.chat(messages=messages)
        # SDK returns an OpenAI-like chat structure in many examples
        content = resp.get("choices", [{}])[0].get("message", {}).get("content", "").strip()
        if content and content.count("|") >= 2:
            # Keep only the first line to avoid accidental extra output
            return content.splitlines()[0]
        return fallback
    except Exception:
        return fallback


In [ ]:
## -----------------------------------------------------------
# Process each image with a provider-agnostic vision function
# Expected vision output format:
#   description | category | workout_type
# -----------------------------------------------------------

gym_equipment = []

for i, image in enumerate(images):

    # Call provider-agnostic vision model (stubbed for now)
    response = vision_model_describe(image)
    line = response.strip()

    # Parse expected pipe-delimited format
    parts = [p.strip() for p in line.split("|")]

    if len(parts) != 3:
        raise ValueError(
            f"Unexpected vision output format for {filenames[i]}:\n{line}\n"
            "Expected: description | category | workout_type"
        )

    description, category, workout_type = parts

    # Store structured result
    gym_equipment.append({
        "filename": filenames[i],
        "description": description,
        "category": category,
        "workout_type": workout_type
    })

# Convert results to DataFrame for downstream use
equipment_df = pd.DataFrame(gym_equipment)

equipment_df

In [ ]:
# -----------------------------------------------------------
# Cell 6: Convert detected equipment into a Pandas DataFrame
# -----------------------------------------------------------
# At this point, `gym_equipment` is a Python list of dictionaries.
# Example:
# [
#   {
#     "filename": "equip1.png",
#     "description": "rack of dumbbells",
#     "category": "dumbbell",
#     "workout_type": "strength"
#   },
#   ...
# ]
#
# Pandas DataFrames make it easy to:
#   - display structured data neatly
#   - sort, filter, and analyze information
#   - use tabular data to feed other agents or logic
#
# Here we convert our list of dicts → DataFrame and show it.
# -----------------------------------------------------------

# Convert the list of dictionaries to a DataFrame
df_equipment = pd.DataFrame(gym_equipment)

# Display the table visually in the notebook
# (In Jupyter, the last line of a cell is automatically rendered.)
df_equipment


In [ ]:
import json


In [ ]:
# -----------------------------------------------------------
# Cell 7: Define the USER PROFILE (who the plan is for)
# -----------------------------------------------------------
# The workout plan depends heavily on:
#   - age
#   - fitness level
#   - training experience
#   - personal goals
#   - available days per week
#   - injuries or limitations
#   - exercise preferences
#
# This block allows students to MODIFY these inputs and see how
# the planner model changes the weekly workout plan.
#
# This is an excellent place for experimentation:
#   - Change goals ("weight loss", "hypertrophy", "flexibility")
#   - Change availability (2 vs 5 days per week)
#   - Add injuries ("knee pain", "no overhead lifting")
#   - Try different preference sets
#
# The planner agent (Granite) will adapt its plan accordingly.
# -----------------------------------------------------------

user_profile = {
    "name": "Alex",                         # User's name (only for display)
    "age": 35,                              # Age can influence exercise intensity
    "gender": "unspecified",                # Optional metadata

    # Beginner / intermediate / advanced
    # This will affect volume, exercise difficulty, and selection.
    "fitness_level": "beginner",

    # Experience with lifting influences safety + complexity of movements.
    "experience_with_strength_training": "low",  # none / low / moderate / high

    # A list of goals; the model will integrate these into the plan structure.
    "primary_goals": [
        "build strength",
        "improve general fitness"
    ],

    # Number of workout days the plan should cover.
    "days_per_week_available": 3,

    # A realistic session length helps ensure the plan isn’t too long.
    "session_length_minutes": 45,

    # List any injuries—model avoids unsafe exercises when possible.
    "injuries_or_limitations": [],

    # Personal likes and dislikes guide exercise selection.
    "exercise_preferences": {
        "likes": ["free weights"],
        "dislikes": ["high impact jumping"]
    }
}

# -----------------------------------------------------------
# Convert both the user profile and equipment list into JSON
# -----------------------------------------------------------
# The planner model receives these values AS TEXT in the prompt.
# JSON formatting ensures:
#   - clean structure
#   - predictable formatting
#   - easier parsing for LLMs
# -----------------------------------------------------------

user_profile_json = json.dumps(user_profile, indent=2)
equipment_json = json.dumps(gym_equipment, indent=2)

# (Optional for debugging)
print("User profile (JSON):")
print(user_profile_json)

print("\nEquipment list (JSON):")
print(equipment_json)


In [ ]:
# -----------------------------------------------------------
# Cell: Generate Workout Plan (IBM watsonx.ai + fallback)
# -----------------------------------------------------------
# This cell:
# 1. Builds a structured planner prompt
# 2. Sends it to watsonx.ai (if configured)
# 3. Otherwise falls back to the original stub output
# 4. Stores the raw table output in `plan_text`
# -----------------------------------------------------------

def text_model_generate(prompt: str) -> str:
    """Generate a weekly plan.

    Must return a pipe-delimited table with EXACT columns:
    Day | Focus | Exercise | Equipment | Sets | RepsOrTime |
    TargetMuscle | Intensity | Warmup | Cooldown | DayNotes

    If watsonx.ai credentials are not provided, falls back to the original
    stub output so the notebook output stays the same.
    """
    fallback = """Day | Focus | Exercise | Equipment | Sets | RepsOrTime | TargetMuscle | Intensity | Warmup | Cooldown | DayNotes
Monday | Upper Body | Push Ups | Bodyweight | 3 | 12 | Chest | Moderate | Arm circles | Stretch chest | Maintain controlled movement
Monday | Upper Body | Dumbbell Rows | Dumbbells | 3 | 10 | Back | Moderate | Arm circles | Stretch lats | Keep spine neutral
Wednesday | Lower Body | Squats | Bodyweight | 3 | 15 | Legs | Moderate | Leg swings | Hamstring stretch | Keep knees aligned
Friday | Cardio | Jump Rope | Rope | 3 | 60 seconds | Full Body | High | Light jog | Deep breathing | Stay hydrated
"""

    if not _watsonx_ready():
        return fallback

    try:
        from ibm_watsonx_ai import Credentials
        from ibm_watsonx_ai.foundation_models import ModelInference
        from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
        from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
    except Exception:
        return fallback

    credentials = Credentials(url=WATSONX_URL, api_key=WATSONX_APIKEY)
    model = ModelInference(
        model_id=WATSONX_TEXT_MODEL,
        credentials=credentials,
        project_id=WATSONX_PROJECT_ID,
        params={
            GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
            GenParams.MAX_NEW_TOKENS: 600,
            GenParams.TEMPERATURE: 0.2,
        },
    )

    try:
        out = model.generate_text(prompt=prompt)
        # SDK commonly returns a string; if dict, try common fields.
        if isinstance(out, str):
            text = out
        elif isinstance(out, dict):
            text = (
                out.get("results", [{}])[0].get("generated_text")
                or out.get("generated_text")
                or ""
            )
        else:
            text = str(out)

        text = (text or "").strip()
        # Basic sanity check: must contain the header and at least one row
        if text.lower().startswith("day |") and "\n" in text:
            return text
        return fallback
    except Exception:
        return fallback


# -----------------------------------------------------------
# Build planner prompt using user profile + equipment
# -----------------------------------------------------------

planner_prompt = f"""
You are an experienced, safety-conscious personal trainer.

Create a structured weekly workout plan using:

USER PROFILE:
{user_profile_json}

AVAILABLE EQUIPMENT:
{equipment_json}

OUTPUT REQUIREMENTS:
- Return ONLY a pipe-delimited table.
- First row must be the header.
- Use this EXACT column order:
  Day | Focus | Exercise | Equipment | Sets | RepsOrTime | TargetMuscle | Intensity | Warmup | Cooldown | DayNotes
- Do not include explanations.
- Do not use '|' inside any field value.
"""

# -----------------------------------------------------------
# Generate the workout plan
# -----------------------------------------------------------

plan_text = text_model_generate(planner_prompt)

# Display raw output for debugging/verification
print("\n✅ Generated Workout Plan:\n")
print(plan_text)

# Safety check to ensure plan_text exists and is valid
if not isinstance(plan_text, str) or not plan_text.strip():
    raise ValueError("Planner did not return valid text.")


In [ ]:
# -----------------------------------------------------------
# Cell 9: Parse the planner's pipe-separated text into a table
# -----------------------------------------------------------
# Goal:
#   Convert the raw text output from the planner model (plan_text)
#   into a structured Pandas DataFrame (df_plan).
#
# The model was instructed to produce lines like:
#
#   Day | Focus | Exercise | Equipment | Sets | RepsOrTime | TargetMuscle | Intensity | Warmup | Cooldown | DayNotes
#   Day 1 | Full body strength | Goblet squat | dumbbell | 3 | 8–10 | legs | moderate | 5 min light cardio | stretching | Focus on good form
#
# Steps:
#   1. Split the text into lines.
#   2. Identify the header line (column names).
#   3. Parse each subsequent line as one exercise row.
#   4. Build a list of dictionaries, then convert to a DataFrame.
# -----------------------------------------------------------

# 1. Split the raw plan text into individual lines.
#    - strip() removes leading/trailing whitespace
#    - we keep only non-empty lines
lines = [l.strip() for l in plan_text.splitlines() if l.strip()]

# If there are no lines at all, something went wrong with the model output.
if not lines:
    raise ValueError("Planner output is empty.")

# 2. The first line is expected to be the header,
#    the remaining lines are data rows.
header_line = lines[0]
data_lines = lines[1:]

# These are the column names we EXPECT based on the prompt.
# The model was told to output columns in this exact order.
expected_cols = [
    "Day", "Focus", "Exercise", "Equipment", "Sets",
    "RepsOrTime", "TargetMuscle", "Intensity",
    "Warmup", "Cooldown", "DayNotes"
]

# 3. Parse the header line by splitting on the '|' separator.
header_parts = [p.strip() for p in header_line.split("|")]
print("Header line parsed as:", header_parts)

# (Optional teaching note)
# Here we could check that `header_parts` exactly matches `expected_cols`.
# For now, we just print it so students can visually verify it.


# This list will hold one dictionary per exercise row.
rows = []

# 4. Process each remaining line as a data row.
for line in data_lines:
    # Split the line into pieces using '|' and strip extra spaces.
    parts = [p.strip() for p in line.split("|")]

    # If the number of fields doesn't match the expected number of columns,
    # we consider the line malformed and skip it.
    # (This keeps the DataFrame clean, but may drop badly formatted lines.)
    if len(parts) != len(expected_cols):
        print("⚠️ Skipping malformed line:", line)
        continue

    # Zip together column names with the values for this row,
    # then turn that into a dictionary.
    row = dict(zip(expected_cols, parts))

    # Add the row dictionary to our list.
    rows.append(row)

# 5. Convert the list of row dicts into a Pandas DataFrame.
df_plan = pd.DataFrame(rows)

print("\n✅ Parsed workout plan into DataFrame:")
df_plan  # Display the table in the notebook


In [ ]:
plan_text


In [ ]:
# -----------------------------------------------------------
# Cell 10: Parse the workout plan → Create DataFrame → Build HTML report
# -----------------------------------------------------------
# This is the final “assembly” cell. It does FOUR major things:
#
# 1.  SAFELY parse the raw planner output (plan_text)
#     The model returns plain text, so we:
#        - split into lines
#        - detect the header row
#        - detect the exercise rows
#        - handle formatting mistakes
#
# 2.  Convert the structured rows into a Pandas DataFrame
#     This becomes df_plan, which is our final “clean table”.
#
# 3.  Build a readable HTML report with:
#        - a header block (user profile summary)
#        - a section for each day with an exercise table
#
# 4.  Save the final HTML file ("workout_plan.html") to disk
#     You can open it in any browser → Print → Save as PDF.
# -----------------------------------------------------------

import pandas as pd
import os

# -----------------------------
# 1) Parse planner output (plan_text) into df_plan
# -----------------------------

# Safety check: make sure the planner produced real text
if not isinstance(plan_text, str) or not plan_text.strip():
    raise ValueError(
        "plan_text is empty or not a string.\n"
        "Make sure the planner_model cell ran correctly and returned text."
    )

# Split output into individual non-empty lines
lines = [l.strip() for l in plan_text.splitlines() if l.strip()]

# Keep only lines that contain the pipe character '|'
# This ensures we ignore any accidental text the model might add.
table_lines = [l for l in lines if "|" in l]

if not table_lines:
    raise ValueError(
        "No table-like lines found in the plan_text.\n"
        "Check the raw planner output above."
    )

# The first line is the header with column names
header_line = table_lines[0]

# Everything after the header is a data row
data_lines = table_lines[1:]

# Expected column names (exact order)
expected_cols = [
    "Day", "Focus", "Exercise", "Equipment", "Sets",
    "RepsOrTime", "TargetMuscle", "Intensity",
    "Warmup", "Cooldown", "DayNotes"
]

# Show the parsed header so students can understand what’s going on
header_parts = [p.strip() for p in header_line.split("|")]
print("Parsed header:", header_parts)

rows = []

# -----------------------------
# Parse each line of the table
# -----------------------------
for line in data_lines:
    parts = [p.strip() for p in line.split("|")]

    # If too many columns → merge extras into the last one
    if len(parts) > len(expected_cols):
        parts = parts[:len(expected_cols)-1] + [" ".join(parts[len(expected_cols)-1:])]

    # If too few columns → pad with empty fields
    if len(parts) < len(expected_cols):
        parts += [""] * (len(expected_cols) - len(parts))

    # If any mismatch remains → skip the line
    if len(parts) != len(expected_cols):
        print("⚠️ Skipping malformed line:", line)
        continue

    # Build a dictionary: {"Day": "..", "Focus": "..", ...}
    row = dict(zip(expected_cols, parts))
    rows.append(row)

# Convert to DataFrame
df_plan = pd.DataFrame(rows)

# Remove completely empty rows (if any slipped in)
df_plan = df_plan.replace("", pd.NA).dropna(how="all").fillna("")

print("\n✅ Parsed workout plan into DataFrame:")
print(df_plan)
print("Shape:", df_plan.shape)

if df_plan.empty:
    raise ValueError("df_plan is empty. The plan text may not have followed the expected table format.")


# -----------------------------------------------------------
# 2) Build the HTML report (header + daily tables)
# -----------------------------------------------------------

# Remove accidental whitespace in column names
df_plan = df_plan.rename(columns=lambda c: c.strip() if isinstance(c, str) else c)

print("\nFinal df_plan columns:", list(df_plan.columns))

# Create a header block summarizing the user's profile
goals_text = ", ".join(user_profile.get("primary_goals", []))

header_html = f"""
<h1>Personalized Workout Plan</h1>
<p><strong>Name:</strong> {user_profile.get("name", "N/A")}</p>
<p><strong>Age:</strong> {user_profile.get("age", "N/A")}</p>
<p><strong>Gender:</strong> {user_profile.get("gender", "N/A")}</p>
<p><strong>Fitness Level:</strong> {user_profile.get("fitness_level", "N/A")}</p>
<p><strong>Goals:</strong> {goals_text}</p>
<p><strong>Days/Week Available:</strong> {user_profile.get("days_per_week_available", "N/A")}</p>
<p><strong>Session Length:</strong> {user_profile.get("session_length_minutes", "N/A")} minutes</p>
"""

if user_profile.get("injuries_or_limitations"):
    header_html += (
        f"<p><strong>Injuries / Limitations:</strong> "
        f"{', '.join(user_profile['injuries_or_limitations'])}</p>"
    )

# Start the HTML output
html_output = [header_html, "<hr>"]

# Columns to show in each day's exercise table
preferred_cols = [
    "Exercise", "Equipment", "Sets", "RepsOrTime", "TargetMuscle", "Intensity"
]

# Safety check: ensure we have a valid Day column
if "Day" not in df_plan.columns:
    df_plan["Day"] = "Plan"

# Loop through each day and build its section
for day_name in df_plan["Day"].unique():
    day_df = df_plan[df_plan["Day"] == day_name].copy()

    focus = day_df["Focus"].iloc[0] if "Focus" in day_df.columns else ""
    warmup = day_df["Warmup"].iloc[0] if "Warmup" in day_df.columns else ""
    cooldown = day_df["Cooldown"].iloc[0] if "Cooldown" in day_df.columns else ""
    notes = day_df["DayNotes"].iloc[0] if "DayNotes" in day_df.columns else ""

    # Keep only the columns we want to show (if they exist)
    display_cols = [c for c in preferred_cols if c in day_df.columns]

    html_output.append(f"<h2>{day_name}</h2>")
    
    if focus:
        html_output.append(f"<p><strong>Focus:</strong> {focus}</p>")
    if warmup:
        html_output.append(f"<p><strong>Warmup:</strong> {warmup}</p>")

    # Insert table of exercises
    if display_cols:
        html_output.append(day_df[display_cols].to_html(index=False, border=1))
    else:
        html_output.append(day_df.to_html(index=False, border=1))

    if cooldown:
        html_output.append(f"<p><strong>Cooldown:</strong> {cooldown}</p>")
    if notes:
        html_output.append(f"<p><strong>Notes:</strong> {notes}</p><br>")

# Join all blocks into a single HTML document
full_html = "\n".join(html_output)

# Save the HTML to disk
output_path = "workout_plan1.html"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(full_html)

print(f"\n✅ Saved HTML to: {os.path.abspath(output_path)}")
print("Open this file in a web browser, then go to: File → Print → Save as PDF")
from IPython.display import HTML, display
display(HTML(full_html))